# WEEK 4:

In [4]:
import pandas as pd
import numpy as np
import time
from sklearn.cluster import KMeans

#!pip install requests
#!pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests
import geocoder # import geocoder
!pip install geopy
!pip install folium
from geopy.geocoders import Nominatim 
import folium # map rendering library

import matplotlib.cm as cm
import matplotlib.colors as colors

### Scrape the Table on the given Wikipedia Page with Beautiful Soup and Display the head of the DataFrame

In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
headers = {'User-Agent': 'Mozilla/5.0'}
page = requests.get(url, headers=headers)

In [6]:
soup = BeautifulSoup(page.text, "html.parser")
table = soup.find('table')
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df=pd.DataFrame(l, columns=["Postcode", "Borough", "Neighbourhood"])
df=df[df['Borough']!='Not assigned'].dropna()
df['Neighbourhood']=df['Neighbourhood'].str.replace('\n','')

In [7]:
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


### Group together rows with the same postalcode as required

In [8]:
df=df.groupby(['Postcode', 'Borough']).agg(lambda col: ', '.join(col)).reset_index()

In [9]:
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


### Fill 'Not- assigned'-values in Neighbourhood-Column with Borough-Name and display the changed rows

In [10]:
na_idx=df[df.Neighbourhood.str.contains('Not assigned')].index
df.loc[na_idx,'Neighbourhood']=df.loc[na_idx,'Borough']
df.iloc[na_idx]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


### Display Shape of Dataframe

In [11]:
df.shape

(103, 3)

### Filter on Toronto Boroughs only

In [12]:
toronto_df=df[df.Borough.str.contains('Toronto')]

In [13]:
toronto_df

,Postcode,Borough,Neighbourhood
37,M4E,East Toronto,The Beaches
41,M4K,East Toronto,"The Danforth West, Riverdale"
42,M4L,East Toronto,"The Beaches West, India Bazaar"
43,M4M,East Toronto,Studio District
44,M4N,Central Toronto,Lawrence Park
45,M4P,Central Toronto,Davisville North
46,M4R,Central Toronto,North Toronto West
47,M4S,Central Toronto,Davisville
48,M4T,Central Toronto,"Moore Park, Summerhill East"
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."


### Test geocoder for 1 Postcode and try at most for 30 seconds to get a response

In [14]:
#initialize start_time
start_time=time.time()

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while((lat_lng_coords is None) and ((time.time()-start_time)<30)):
    g = geocoder.google('{}, Toronto, Ontario'.format('M5A'))
    lat_lng_coords = g.latlng

if (lat_lng_coords is None):
    print('Geocoder not responding in time: use CSV')
else:
    print('Geocoder request succeeded, coordinates are set')
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

Geocoder not responding in time: use CSV


### Use CSV since Geocoder does not reply

In [15]:
coordinates=pd.read_csv('Geospatial_Coordinates.csv')

In [16]:
toronto_df=pd.merge(toronto_df, coordinates, left_on='Postcode', right_on='Postal Code').drop('Postal Code', axis=1)

In [17]:
toronto_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


## For the rest of the notebook we will follow exactly the exploration and clustering steps from the New York Example only for Toronto

## Explore venues

In [22]:
CLIENT_ID = 'NNO2TDZ2XO2YHCMISLHPTUFBAK0YDZBX3RTF04YY4Q5FVR1W' # your Foursquare ID
CLIENT_SECRET = 'UTNENPNX0IFTQP1VPKXI035AFXQEC5XJCN0XJ5YRMYB05HAH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NNO2TDZ2XO2YHCMISLHPTUFBAK0YDZBX3RTF04YY4Q5FVR1W
CLIENT_SECRET: UTNENPNX0IFTQP1VPKXI035AFXQEC5XJCN0XJ5YRMYB05HAH


### Get Venues Nearby

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=20):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(623, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


### Count Venues per Neighborhood

In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",20,20,20,20,20,20
Berczy Park,20,20,20,20,20,20
"Brockton, Exhibition Place, Parkdale Village",20,20,20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",20,20,20,20,20,20
Central Bay Street,20,20,20,20,20,20
"Chinatown, Grange Park, Kensington Market",20,20,20,20,20,20
Christie,16,16,16,16,16,16


### Count Venue Categories

In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 162 uniques categories.


### Find Most Common Venues

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
toronto_onehot.shape

(623, 162)

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,"Adelaide, King, Richmond",0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.050000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.000000,0.00,0.000000
1,Berczy Park,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.00,...,0.05,0.05,0.00,0.00,0.00,0.00,0.05,0.000000,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0500,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.0625,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
6,Central Bay Street,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.00,...,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.100000,0.05,0.000000
8,Christie,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
9,Church and Wellesley,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.00,...,0.05,0.00,0.00,0.05,0.00,0.00,0.00,0.000000,0.00,0.000000


In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Asian Restaurant,Coffee Shop,Gym / Fitness Center,Seafood Restaurant,Bar,Speakeasy,Hotel,Plaza,Café
1,Berczy Park,Farmers Market,Basketball Stadium,Concert Hall,Seafood Restaurant,Bistro,Beer Bar,Cocktail Bar,Bakery,Steakhouse,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Yoga Studio,Bakery,Furniture / Home Store,Grocery Store,Gym,Italian Restaurant,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Spa,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Burrito Place,Recording Studio,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Plane,Sculpture Garden,Bar,Coffee Shop,Airport


## Clustering

In [33]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df
toronto_merged = toronto_merged.rename(columns={'Neighbourhood': 'Neighborhood'})

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Other Great Outdoors,Trail,Pub,Health Food Store,Wine Shop,Convenience Store,Diner,Dessert Shop,Deli / Bodega,Dance Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pub,Juice Bar,Brewery,Dessert Shop,Restaurant,Cosmetics Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Fast Food Restaurant,Fish & Chips Shop,Burger Joint,Italian Restaurant,Burrito Place,Ice Cream Shop,Pub,Steakhouse,Liquor Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Gay Bar,Sandwich Place,Italian Restaurant,Stationery Store,Ice Cream Shop,Fish Market,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Construction & Landscaping,Bus Line,Swim School,Wine Shop,Creperie,Discount Store,Diner,Dessert Shop,Deli / Bodega


In [35]:
toronto_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Other Great Outdoors,Trail,Pub,Health Food Store,Wine Shop,Convenience Store,Diner,Dessert Shop,Deli / Bodega,Dance Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pub,Juice Bar,Brewery,Dessert Shop,Restaurant,Cosmetics Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Fast Food Restaurant,Fish & Chips Shop,Burger Joint,Italian Restaurant,Burrito Place,Ice Cream Shop,Pub,Steakhouse,Liquor Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Gay Bar,Sandwich Place,Italian Restaurant,Stationery Store,Ice Cream Shop,Fish Market,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Construction & Landscaping,Bus Line,Swim School,Wine Shop,Creperie,Discount Store,Diner,Dessert Shop,Deli / Bodega
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Park,Gym,Dog Run,Clothing Store,Sandwich Place,Hotel,Food & Drink Shop,Breakfast Spot,Pizza Place,Cuban Restaurant
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Café,Diner,Spa,Salon / Barbershop,Restaurant,Burger Joint
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Dessert Shop,Sushi Restaurant,Gym,Italian Restaurant,Diner,Sandwich Place,Brewery,Indian Restaurant,Café,Seafood Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Playground,Gym,Restaurant,Wine Shop,Convenience Store,Diner,Dessert Shop,Deli / Bodega,Dance Studio,Cuban Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,1,Pub,Coffee Shop,Liquor Store,Pizza Place,Supermarket,Sushi Restaurant,Sports Bar,Bagel Shop,American Restaurant,Restaurant


In [36]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


/home/bardiya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [43]:
i=0
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pub,Juice Bar,Brewery,Dessert Shop,Restaurant,Cosmetics Shop
2,East Toronto,0,Park,Fast Food Restaurant,Fish & Chips Shop,Burger Joint,Italian Restaurant,Burrito Place,Ice Cream Shop,Pub,Steakhouse,Liquor Store
4,Central Toronto,0,Park,Construction & Landscaping,Bus Line,Swim School,Wine Shop,Creperie,Discount Store,Diner,Dessert Shop,Deli / Bodega
5,Central Toronto,0,Park,Gym,Dog Run,Clothing Store,Sandwich Place,Hotel,Food & Drink Shop,Breakfast Spot,Pizza Place,Cuban Restaurant
7,Central Toronto,0,Dessert Shop,Sushi Restaurant,Gym,Italian Restaurant,Diner,Sandwich Place,Brewery,Indian Restaurant,Café,Seafood Restaurant
10,Downtown Toronto,0,Park,Trail,Playground,Building,Cosmetics Shop,Diner,Dessert Shop,Deli / Bodega,Dance Studio,Cuban Restaurant


In [44]:
i=1
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Other Great Outdoors,Trail,Pub,Health Food Store,Wine Shop,Convenience Store,Diner,Dessert Shop,Deli / Bodega,Dance Studio
3,East Toronto,1,Café,Coffee Shop,Bakery,Gay Bar,Sandwich Place,Italian Restaurant,Stationery Store,Ice Cream Shop,Fish Market,Seafood Restaurant
6,Central Toronto,1,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Café,Diner,Spa,Salon / Barbershop,Restaurant,Burger Joint
9,Central Toronto,1,Pub,Coffee Shop,Liquor Store,Pizza Place,Supermarket,Sushi Restaurant,Sports Bar,Bagel Shop,American Restaurant,Restaurant
11,Downtown Toronto,1,Restaurant,Café,Japanese Restaurant,Caribbean Restaurant,Deli / Bodega,Jewelry Store,Italian Restaurant,Indian Restaurant,Bakery,Butcher
12,Downtown Toronto,1,Japanese Restaurant,Hobby Shop,Mexican Restaurant,Park,Pizza Place,Pub,Ramen Restaurant,Juice Bar,Burger Joint,Restaurant
13,Downtown Toronto,1,Coffee Shop,Bakery,Breakfast Spot,Gym / Fitness Center,Pub,Historic Site,Spa,Farmers Market,Restaurant,Dessert Shop
14,Downtown Toronto,1,Café,Music Venue,Taco Place,Beer Bar,Diner,Ramen Restaurant,Burger Joint,Plaza,Burrito Place,Pizza Place
15,Downtown Toronto,1,Gastropub,Coffee Shop,Japanese Restaurant,Restaurant,Cosmetics Shop,Italian Restaurant,Middle Eastern Restaurant,Café,Hotel,BBQ Joint
16,Downtown Toronto,1,Farmers Market,Basketball Stadium,Concert Hall,Seafood Restaurant,Bistro,Beer Bar,Cocktail Bar,Bakery,Steakhouse,Breakfast Spot


In [40]:
i=2
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,2,Playground,Gym,Restaurant,Wine Shop,Convenience Store,Diner,Dessert Shop,Deli / Bodega,Dance Studio,Cuban Restaurant


In [41]:
i=3
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,3,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Creperie,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega


In [42]:
i=4
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,4,Garden,Wine Shop,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie
